In [2]:
from collections import defaultdict

In [3]:
from itertools import islice
with open("wikicorpus_01") as myfile:
    head = list(islice(myfile, 651))

In [4]:
sent = []
for s in head:
    sent.append(s.split())

In [5]:
words = []
sentences = []
for s in sent:
    if len(s) == 4 and s[0] not in {'(', ')', ','} and '[' not in s[1]:
        words.append((s[0], s[1], s[2], s[3]))
        #if 'endofarticle' in s[1]:
        #    sentences.append(words)
        #    words = []

In [6]:
words[:4]

[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
 ('La', 'el', 'DA0FS0', '0'),
 ('Plata', 'plata', 'NP00000', '10501473'),
 ('Buenos_Aires', 'buenos_aires', 'NP00000', '0')]

In [7]:
count = dict()
for fst,snd,trd, fth in words:
    if fst not in count:
        count[fst] = 1
    else:
        count[fst] += 1

In [8]:
aux = []
check = []
for fst,snd,trd, fth in words:
    aux.append((fst,snd,trd, fth))
    if fst == '.':
        check.append(aux)
        aux = []

In [9]:
check[:2]

[[('Néstor_Gabriel_Martinena', 'néstor_gabriel_martinena', 'NP00000', '0'),
  ('La', 'el', 'DA0FS0', '0'),
  ('Plata', 'plata', 'NP00000', '10501473'),
  ('Buenos_Aires', 'buenos_aires', 'NP00000', '0'),
  ('Argentina', 'argentina', 'NP00000', '02035234'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('un', 'uno', 'DI0MS0', '0'),
  ('futbolista', 'futbolista', 'NCCS000', '0'),
  ('argentino', 'argentino', 'AQ0MS0', '00278090'),
  ('.', '.', 'Fp', '0')],
 [('Juega', 'jugar', 'VMM02S0', '00727813'),
  ('de', 'de', 'SPS00', '0'),
  ('delantero', 'delantero', 'NCMS000', '00466114'),
  ('y', 'y', 'CC', '0'),
  ('su', 'su', 'DP3CS0', '0'),
  ('equipo', 'equipo', 'NCMS000', '06093198'),
  ('actual', 'actual', 'AQ0CS0', '01667781'),
  ('es', 'ser', 'VSIP3S0', '01775973'),
  ('Gimnasia', 'gimnasia', 'NP00000', '00275488'),
  ('y', 'y', 'CC', '0'),
  ('Esgrima_La_Plata_de_la_Primera_División_de_Argentina',
   'esgrima_la_plata_de_la_primera_división_de_argentina',
   'NP00000',
   '0'),
  ('.', '.'

In [10]:
dicc_backward = defaultdict(list)
dicc_forward = defaultdict(list)
for s in check:
    for i,w in enumerate(s):
        if i != 0:
            dicc_backward[w[0]].append((s[i-1][0], s[i-1][1], s[i-1][2], s[i-1][3]))
        else:
            dicc_backward[w[0]].append(('[W.Start]', '[L.Start]', '[T.Start]', '[S.Start]'))
        if i != len(s)-1:
            dicc_forward[w[0]].append((s[i+1][0], s[i+1][1], s[i+1][2], s[i+1][3]))
        else:
            dicc_forward[w[0]].append(('[W.End]', '[T.End]', '[T.End]', '[S.End]'))

In [11]:
dicc_backward['equipo']

[('su', 'su', 'DP3CS0', '0'),
 ('el', 'el', 'DA0MS0', '0'),
 ('su', 'su', 'DP3CS0', '0'),
 ('el', 'el', 'DA0MS0', '0')]

In [12]:
dicc_forward['equipo']

[('actual', 'actual', 'AQ0CS0', '01667781'),
 ('Racing_Club', 'racing_club', 'NP00000', '0'),
 ('y', 'y', 'CC', '0'),
 ('de', 'de', 'SPS00', '0')]

In [13]:
count['La']

1

In [14]:
dicc = dict()
vec = dict()
for fst,snd,trd,fth in words:
    features = defaultdict(int)
    if snd not in dicc and snd not in {'.', ',', ':', ';'} and snd.isalpha() and count[fst] > 2:
        dicc[fst] = features
        vec[fst] = trd
        features[fth] = 1 #synset
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[3]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[3]+"+1"] += 1
        
    elif fst in dicc:
        has_it = dicc[fst]
        if fth in has_it:
            has_it[fth] += 1
        else:
            has_it[fth] = 1

In [15]:
dicc['cuyo']

defaultdict(int,
            {'0': 3,
             '0-1': 3,
             '04538868+1': 1,
             '06239374+1': 1,
             '07076177+1': 1,
             'Ayuntamiento_de_Valle_de_Losa-1': 1,
             'Ha-1': 1,
             'La_Zarza-1': 1,
             'plan+1': 1,
             'promotor+1': 1,
             'sistema+1': 1})

In [16]:
vec

{'a': 'SPS00',
 'alturas': 'NCFP000',
 'con': 'SPS00',
 'cuyo': 'PR0MS000',
 'de': 'SPS00',
 'el': 'DA0MS0',
 'en': 'SPS00',
 'equipo': 'NCMS000',
 'es': 'VSIP3S0',
 'localidad': 'NCFS000',
 'minutos': 'NCMP000',
 'partido': 'NCMS000',
 'planta': 'NCFS000',
 'por': 'SPS00',
 'que': 'PR0CN000',
 'su': 'DP3CS0',
 'tiempo': 'NCMS000',
 'una': 'DI0FS0',
 'y': 'CC'}

In [17]:
matrix = []
vector = []
final_check = []
for key, value in dicc.items():    #preparo la lista de dicts y la lista de chequeo para recuperar el nombre al final
    final_check.append(key)
    matrix.append(value)
for key, value in vec.items():       #vector de synsets
    vector.append(value)

In [18]:
vector

['VSIP3S0',
 'SPS00',
 'CC',
 'DP3CS0',
 'NCMS000',
 'SPS00',
 'DA0MS0',
 'SPS00',
 'NCMP000',
 'NCMS000',
 'NCMS000',
 'PR0CN000',
 'SPS00',
 'DI0FS0',
 'NCFS000',
 'SPS00',
 'PR0MS000',
 'NCFS000',
 'NCFP000']

In [19]:
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer()
X = v.fit_transform(matrix)

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [25]:
Y = SelectKBest(chi2, k=4).fit_transform(X, vector)

In [26]:
Y.shape

(19, 4)

In [27]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(Y)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [28]:
labels = km.predict(Y)

In [30]:
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(final_check[i])
print(clusters)

defaultdict(<class 'set'>, {0: {'su', 'equipo', 'planta', 'una', 'que', 'con', 'es', 'por', 'partido', 'tiempo', 'alturas', 'localidad', 'minutos', 'cuyo'}, 2: {'de'}, 1: {'el', 'a', 'y', 'en'}})
